   {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
         "## 5. Optimización del Modelo\n",
         "\n",
         "Optimizaremos el mejor modelo de regresión utilizando GridSearchCV para encontrar los hiperparámetros óptimos."
      ]
   },
   {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
         "# Optimizar el mejor modelo de regresión\n",
         "mejor_nombre = mejor_modelo_regresion[0]\n",
         "mejor_modelo = mejor_modelo_regresion[1]['modelo']\n",
         "\n",
         "print(f\"Optimizando {mejor_nombre}...\")\n",
         "\n",
         "# Definir parámetros de búsqueda según el tipo de modelo\n",
         "if mejor_nombre == 'Regresión Lineal':\n",
         "    param_grid = {\n",
         "        'fit_intercept': [True, False],\n",
         "        'normalize': [True, False],\n",
         "        'copy_X': [True, False]\n",
         "    }\n",
         "elif mejor_nombre == 'Ridge':\n",
         "    param_grid = {\n",
         "        'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],\n",
         "        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']\n",
         "    }\n",
         "elif mejor_nombre == 'Lasso':\n",
         "    param_grid = {\n",
         "        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],\n",
         "        'selection': ['cyclic', 'random']\n",
         "    }\n",
         "elif mejor_nombre == 'Random Forest':\n",
         "    param_grid = {\n",
         "        'n_estimators': [50, 100, 200],\n",
         "        'max_depth': [None, 10, 20, 30],\n",
         "        'min_samples_split': [2, 5, 10],\n",
         "        'min_samples_leaf': [1, 2, 4]\n",
         "    }\n",
         "elif mejor_nombre == 'Gradient Boosting':\n",
         "    param_grid = {\n",
         "        'n_estimators': [50, 100, 200],\n",
         "        'learning_rate': [0.01, 0.1, 0.2],\n",
         "        'max_depth': [3, 5, 7],\n",
         "        'min_samples_split': [2, 5, 10]\n",
         "    }\n",
         "\n",
         "# Realizar búsqueda de cuadrícula\n",
         "grid_search = GridSearchCV(estimator=mejor_modelo, param_grid=param_grid, \n",
         "                          cv=5, scoring='r2', n_jobs=-1)\n",
         "grid_search.fit(X_train, y_train)\n",
         "\n",
         "print(f\"Mejores parámetros: {grid_search.best_params_}\")\n",
         "print(f\"Mejor puntuación R²: {grid_search.best_score_:.4f}\")\n",
         "\n",
         "# Evaluar el modelo optimizado\n",
         "modelo_optimizado = grid_search.best_estimator_\n",
         "modelo_opt, r2_opt = evaluar_modelo_regresion(modelo_optimizado, X_train, X_test, y_train, y_test, \n",
         "                                             f\"{mejor_nombre} (Optimizado)\")\n",
         "\n",
         "# Comparar con el modelo original\n",
         "mejora = r2_opt - mejor_modelo_regresion[1]['r2']\n",
         "print(f\"\\nMejora en R²: {mejora:.4f} ({mejora*100:.2f}%)\")"
      ]
   },
  {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
         "## 6. Conclusiones y Recomendaciones\n",
         "\n",
         "Basándonos en los resultados del modelado y análisis, podemos extraer las siguientes conclusiones y recomendaciones:"
      ]
  },
  {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
         "# Función para generar recomendaciones basadas en los resultados del modelo\n",
         "def generar_recomendaciones(modelo_regresion, modelo_clustering, df, user_profiles):\n",
         "    # 1. Identificar categorías con calificaciones más bajas\n",
         "    categorias_bajas = df.groupby('categoria')['calificacion'].mean().sort_values().head(3)\n",
         "    \n",
         "    # 2. Identificar factores que más influyen en las calificaciones (del modelo de regresión)\n",
         "    if hasattr(modelo_regresion, 'feature_importances_'):\n",
         "        importancias = pd.DataFrame({\n",
         "            'caracteristica': X_train.columns,\n",
         "            'importancia': modelo_regresion.feature_importances_\n",
         "        }).sort_values('importancia', ascending=False)\n",
         "        factores_importantes = importancias.head(5)\n",
         "    elif hasattr(modelo_regresion, 'coef_'):\n",
         "        coefs = pd.DataFrame({\n",
         "            'caracteristica': X_train.columns,\n",
         "            'coeficiente': modelo_regresion.coef_\n",
         "        })\n",
         "        factores_importantes = coefs.reindex(coefs['coeficiente'].abs().sort_values(ascending=False).index).head(5)\n",
         "    \n",
         "    # 3. Analizar perfiles de usuario (del modelo de clustering)\n",
         "    perfiles = user_profiles.groupby('cluster').mean()\n",
         "    \n",
         "    # Generar recomendaciones\n",
         "    print(\"\\n=== CONCLUSIONES Y RECOMENDACIONES ===\\n\")\n",
         "    \n",
         "    print(\"1. ÁREAS DE MEJORA:\")\n",
         "    print(\"Las categorías con calificaciones más bajas son:\")\n",
         "    for categoria, valor in categorias_bajas.items():\n",
         "        print(f\"   - {categoria}: {valor:.2f}/10\")\n",
         "    print(\"\\nRecomendación: Enfocar esfuerzos en mejorar estas áreas mediante programas específicos.\")\n",
         "    \n",
         "    print(\"\\n2. FACTORES INFLUYENTES:\")\n",
         "    print(\"Los factores que más influyen en las calificaciones son:\")\n",
         "    for _, row in factores_importantes.iterrows():\n",
         "        if hasattr(modelo_regresion, 'feature_importances_'):\n",
         "            print(f\"   - {row['caracteristica']}: {row['importancia']:.4f} de importancia\")\n",
         "        else:\n",
         "            print(f\"   - {row['caracteristica']}: coeficiente de {row['coeficiente']:.4f}\")\n",
         "    \n",
         "    print(\"\\n3. PERFILES DE USUARIO:\")\n",
         "    print(f\"Se identificaron {len(perfiles)} perfiles distintos de usuarios:\")\n",
         "    \n",
         "    for cluster_id, perfil in perfiles.iterrows():\n",
         "        print(f\"\\nPerfil {cluster_id}:\")\n",
         "        # Mostrar características demográficas\n",
         "        print(f\"   Edad promedio: {perfil['edad']:.1f} años\")\n",
         "        \n",
         "        # Mostrar categorías con mayor y menor puntuación para este perfil\n",
         "        categorias_cols = [col for col in perfil.index if col not in ['edad', 'cluster'] and not col.startswith('sexo_') and not col.startswith('estado_civil_')]\n",
         "        cat_scores = {cat: perfil[cat] for cat in categorias_cols}\n",
         "        mejor_cat = max(cat_scores.items(), key=lambda x: x[1])\n",
         "        peor_cat = min(cat_scores.items(), key=lambda x: x[1])\n",
         "        \n",
         "        print(f\"   Fortaleza: {mejor_cat[0]} ({mejor_cat[1]:.2f}/10)\")\n",
         "        print(f\"   Área de mejora: {peor_cat[0]} ({peor_cat[1]:.2f}/10)\")\n",
         "    \n",
         "    print(\"\\n4. RECOMENDACIONES GENERALES:\")\n",
         "    print(\"   - Implementar programas específicos para las categorías con puntuaciones más bajas\")\n",
         "    print(\"   - Personalizar intervenciones según los perfiles de usuario identificados\")\n",
         "    print(\"   - Realizar seguimiento continuo para evaluar la efectividad de las intervenciones\")\n",
         "    print(\"   - Considerar factores demográficos al diseñar programas de mejora\")\n",
         "\n",
         "# Generar recomendaciones\n",
         "generar_recomendaciones(modelo_opt, kmeans, df, user_profiles)"
      ]
  },
  {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
         "## Resumen del Modelado\n",
         "\n",
         "En este notebook, hemos realizado un análisis completo de los datos de la \"Ruleta de la Vida\":\n",
         "\n",
         "1. **Preprocesamiento de Datos**: Limpiamos y transformamos los datos para su análisis.\n",
         "\n",
         "2. **Análisis Exploratorio**: Identificamos patrones y relaciones en los datos.\n",
         "\n",
         "3. **Modelado Predictivo**: Desarrollamos modelos de regresión para predecir calificaciones.\n",
         "\n",
         "4. **Segmentación de Usuarios**: Utilizamos clustering para identificar perfiles de usuarios.\n",
         "\n",
         "5. **Optimización**: Mejoramos el rendimiento del modelo mediante ajuste de hiperparámetros.\n",
         "\n",
         "6. **Recomendaciones**: Generamos insights accionables basados en los resultados del modelado.\n",
         "\n",
         "Este análisis proporciona una base sólida para la toma de decisiones y el diseño de intervenciones personalizadas para mejorar el bienestar de los usuarios en las diferentes áreas de la vida evaluadas por la \"Ruleta de la Vida\"."
      ]
   }
"metadata": {
   "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
   },
  "language_info": {
      "codemirror_mode": {
      "name": "ipython",
      "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.8.10"
   }
},
 "nbformat": 4,
 "nbformat_minor": 4


